In [1]:
import cv2
import numpy as np

def crop_fisheye_with_overlap(image_path, output_path, direction, size=1024, overlap_factor=0.2):
    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    crop_size = int(size * (1 + overlap_factor))  # e.g., 1228 with 20% overlap
    half_crop = crop_size // 2

    # Shift center based on direction to preserve overlaps
    if direction == "front":  # +Z
        x_shift, y_shift = 0, 0
    elif direction == "back":  # -Z
        x_shift, y_shift = 0, 0  # Opposite view, adjust if fisheye is rotated
    elif direction == "left":  # -X
        x_shift = -int(size * overlap_factor / 2)  # Shift left for front overlap
        y_shift = 0
    elif direction == "right":  # +X
        x_shift = int(size * overlap_factor / 2)   # Shift right
        y_shift = 0

    cropped = img[center[1] - half_crop + y_shift:center[1] + half_crop + y_shift,
                  center[0] - half_crop + x_shift:center[0] + half_crop + x_shift]
    resized = cv2.resize(cropped, (size, size), interpolation=cv2.INTER_AREA)
    cv2.imwrite(output_path, resized)

crop_fisheye_with_overlap("input/front.png", "front_processed.png", "front")
crop_fisheye_with_overlap("input/back.png", "back_processed.png", "back")
crop_fisheye_with_overlap("input/left.png", "left_processed.png", "left")
crop_fisheye_with_overlap("input/right.png", "right_processed.png", "right")

In [6]:
import cv2
import numpy as np

def crop_fisheye_for_skybox(image_path, output_path, direction, size=1024, fov=180):
    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    # 90° out of 180° FOV = half the image width/height
    crop_size = int((90 / fov) * min(h, w))  # e.g., 960 if 1920x1920
    half_crop = crop_size // 2

    # Adjust crop to align edges with neighbors
    if direction == "front":  # +Z
        x_shift, y_shift = 0, 0
    elif direction == "back":  # -Z
        x_shift, y_shift = 0, 0
    elif direction == "left":  # -X
        x_shift = -int(crop_size * 0.1)  # Shift left for front overlap
        y_shift = 0
    elif direction == "right":  # +X
        x_shift = int(crop_size * 0.1)   # Shift right for front overlap
        y_shift = 0

    cropped = img[center[1] - half_crop + y_shift:center[1] + half_crop + y_shift,
                  center[0] - half_crop + x_shift:center[0] + half_crop + x_shift]
    resized = cv2.resize(cropped, (size, size), interpolation=cv2.INTER_AREA)
    cv2.imwrite(output_path, resized)

crop_fisheye_with_overlap("input/front.png", "front_processed.png", "front")
crop_fisheye_with_overlap("input/back.png", "back_processed.png", "back")
crop_fisheye_with_overlap("input/left.png", "left_processed.png", "left")
crop_fisheye_with_overlap("input/right.png", "right_processed.png", "right")

In [16]:
import cv2
import numpy as np

def crop_fisheye_center(image_path, output_path, direction, size=1024, fov=180):
    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    center = (w // 2, h // 2)  # Image center
    
    # Calculate crop size for 90° out of total FOV
    crop_size = int((90 / fov) * min(h, w))  # e.g., 960 for 1920x1920, 180° FOV
    half_crop = crop_size // 2
    
    # No shifts - crop exactly at center
    cropped = img[center[1] - half_crop:center[1] + half_crop,
                  center[0] - half_crop:center[0] + half_crop]
    
    # Check if crop fits within image bounds
    if cropped.shape[0] == 0 or cropped.shape[1] == 0:
        print(f"Error: Crop failed for {image_path}. Check image size or FOV.")
        return
    
    resized = cv2.resize(cropped, (size, size), interpolation=cv2.INTER_AREA)
    cv2.imwrite(output_path, resized)
    print(f"Cropped and saved {output_path}")

# Run for all images
crop_fisheye_with_overlap("input/front.png", "front_processed.png", "front")
crop_fisheye_with_overlap("input/back.png", "ouput/back_processed.png", "back")
crop_fisheye_with_overlap("input/left.png", "ouput/left_processed.png", "left")
crop_fisheye_with_overlap("input/right.png", "ouput/right_processed.png", "right")

In [21]:
import cv2
import numpy as np

def crop_fisheye_with_overlap(image_path, output_path, direction, size=1024, fov=180):
    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    center_x, center_y = w // 2, h // 2
    
    # Crop 120° to include 90° + overlaps
    crop_fov = 120  # 90° core + 15° each side
    crop_size = int((crop_fov / fov) * min(h, w))  # e.g., 1280 for 1920x1920
    half_crop = crop_size // 2
    
    cropped = img[center_y - half_crop:center_y + half_crop,
                  center_x - half_crop:center_x + half_crop]
    resized = cv2.resize(cropped, (size, size), interpolation=cv2.INTER_AREA)
    cv2.imwrite(output_path, resized)

crop_fisheye_with_overlap("input/front.png", "front_processed.png", "front")
crop_fisheye_with_overlap("input/back.png", "back_processed.png", "back")
crop_fisheye_with_overlap("input/left.png", "left_processed.png", "left")
crop_fisheye_with_overlap("input/right.png", "right_processed.png", "right")

In [22]:
import cv2
import numpy as np

def create_top_bottom_from_crops(input_paths, top_output, bottom_output, size=1024, edge_height=341):
    imgs = [cv2.imread(p) for p in input_paths]
    
    # Extract top and bottom from 120°x120° crops (resized to 1024x1024)
    top_edges = [img[0:edge_height, :] for img in imgs]  # +60° to +45° (sky)
    bottom_edges = [img[size-edge_height:size, :] for img in imgs]  # -45° to -60° (ground)
    
    # Blend across all four images
    top_blended = np.mean(top_edges, axis=0).astype(np.uint8)
    bottom_blended = np.mean(bottom_edges, axis=0).astype(np.uint8)
    
    # Already 1024 wide, resize height to 1024
    top_resized = cv2.resize(top_blended, (size, size), interpolation=cv2.INTER_AREA)
    bottom_resized = cv2.resize(bottom_blended, (size, size), interpolation=cv2.INTER_AREA)
    
    cv2.imwrite(top_output, top_resized)
    cv2.imwrite(bottom_output, bottom_resized)
    print(f"Saved {top_output} and {bottom_output}")

inputs = ["front_processed.png", "back_processed.png", "left_processed.png", "right_processed.png"]
create_top_bottom_from_crops(inputs, "top.png", "bottom.png")

Saved top.png and bottom.png
